In [2]:
import itertools
import pandas as pd
from datasets import load_dataset, load_from_disk
import json
from PIL import Image
import os

In [3]:
test_dataset = load_from_disk("/workspace/data/filtered_dataset")

In [4]:
print("Filtered test Dataset:")
print(test_dataset)

print(f"Loaded {len(test_dataset)} image-caption entries\n")

print("📌 First 5 entries:")
first_5 = test_dataset.select(range(5))
for row in first_5:
    print(row, "\n")

print("📋 Column types:")
print(test_dataset.features)

print("\n🔎 Size")
print((len(test_dataset), len(test_dataset.features)))

Filtered test Dataset:
Dataset({
    features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
    num_rows: 10
})
Loaded 10 image-caption entries

📌 First 5 entries:
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x711F0B1B5E10>, 'caption': ['Three Caucasian hikers in summer clothes walk a dirt path through a wooded meadow during the day.', 'Three people walking down a muddy dirt road on a cloudy day.', 'Three people are walking down a dirt road in the country.', 'Three people walking on a trail in the middle of the day.', 'Three people are walking down a dirt path.'], 'sentids': ['12500', '12501', '12502', '12503', '12504'], 'split': 'train', 'img_id': '2500', 'filename': '182169366.jpg'} 

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=356x500 at 0x711F09D264D0>, 'caption': ['A waitress in a black T-shirt and black shorts at Cafe Mambo serves a tray of beverages to a seated customer in a red tank top and black bandanna.'

In [11]:
print("Filtered test Dataset:")
print(test_dataset)

print(f"Loaded {len(test_dataset)} image-caption entries\n")

print("📌 First 5 entries:")
first_5 = test_dataset.select(range(5))
for row in first_5:
    print(row, "\n")

print("📋 Column types:")
print(test_dataset.features)

print("\n🔎 Size")
print((len(test_dataset), len(test_dataset.features)))

Loaded 50 image-caption entries

📌 First 5 entries:
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x461 at 0x7FECFD27A530>, 'caption': ['The man with pierced ears is wearing glasses and an orange hat.', 'A man with glasses is wearing a beer can crocheted hat.', 'A man with gauges and glasses is wearing a Blitz hat.', 'A man in an orange hat starring at something.', 'A man wears an orange hat and glasses.'], 'sentids': ['125', '126', '127', '128', '129'], 'split': 'test', 'img_id': '25', 'filename': '1007129816.jpg'} 

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x333 at 0x7FECFD279FC0>, 'caption': ['A black and white dog is running in a grassy garden surrounded by a white fence.', 'A Boston Terrier is running on lush green grass in front of a white fence.', 'A black and white dog is running through the grass.', 'A dog runs on the green grass near a wooden fence.', 'A Boston terrier is running in the grass.'], 'sentids': ['170', '171', '172', 

In [5]:
from collections import Counter

filename_counts = Counter(test_dataset["filename"])

# Print filenames with more than 1 occurrence
for filename, count in filename_counts.items():
    if count > 1:
        print(f"{filename}: {count} occurrences")

num_duplicates = sum(1 for count in filename_counts.values() if count > 1)
print(f"\nTotal filenames with duplicates: {num_duplicates}")


Total filenames with duplicates: 0


In [6]:
from unsloth import FastLanguageModel
from transformers import AutoProcessor, AutoTokenizer, AutoModelForVision2Seq
import torch

model_id = "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_id,
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth",
)

model = FastLanguageModel.for_inference(model)
processor = AutoProcessor.from_pretrained(model_id)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothGKDTrainer: No module named 'UnslothGKDTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.4.7: Fast Qwen2_5_Vl patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.448 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.97G [00:00<?, ?B/s]

AttributeError: 'dict' object has no attribute 'to_dict'

In [16]:
import torch
import time
from transformers import TextStreamer
import pandas as pd

streamer = TextStreamer(tokenizer, skip_prompt=True)

def run_vlm_inference(prompt: str, filename: str, df: pd.DataFrame):
    """
    Perform inference on a given image (by filename) from the dataframe using a custom prompt.

    Args:
        prompt (str): The prompt text (can include mask tokens)
        filename (str): Filename of the image in the DataFrame
        df (pd.DataFrame): DataFrame with 'filename' and 'image' columns

    Returns:
        Tuple[str, float, float]: (Generated output, inference time in seconds, VRAM used in GB)
    """
    row = df[df["filename"] == filename]
    if row.empty:
        print(f"[ERROR] No image found with filename: {filename}")
        return None, 0.0, 0.0

    row = row.iloc[0]
    image = row["image"]
    caption = row["caption"]

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image", "image": image}
            ]
        }
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(images=image, text=input_text, return_tensors="pt").to("cuda")

    torch.cuda.reset_peak_memory_stats()
    start_mem = torch.cuda.memory_allocated() / 1024 / 1024 / 1024  # in GB
    start_time = time.time()

    print(f"🔹 Image: {filename}")
    print(f"🧾 Prompt: {prompt}")
    print("📤 Output:")
    
    # Perform inference
    output_ids = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=128,
        use_cache=True,
        temperature=1.0,
        top_p=0.95
    )

    end_time = time.time()
    end_mem = torch.cuda.max_memory_allocated() / 1024**3

    time_taken = round(end_time - start_time, 3)
    vram_used = round(end_mem - start_mem, 3)
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print(f"⏱️ Time taken: {time_taken} sec | 🧠 VRAM used: {vram_used} GB")
    print("-" * 80)

    return decoded_output, time_taken, vram_used

In [ ]:
filename = "000404.jpg"

row = df[df["filename"] == filename]
ground_truth = row.iloc[0]["caption"]

inference_outputs = []
vram_usages = []
inference_times = []

prompts = [
    "",  # No Prompt
    "Describe &&damage 12 sedan drive’ this !!image.",  # Noisy
    "An image of a damaged car parked on the side of the road.",  # Hand-crafted
    "You are an insurance claims assessor. Provide a detailed description of the car’s condition.",  # Roleplay
    "This <part_1> of the car has <damage_type_1>. The severity appears to be <severity_1>. Additional notes: <text_1>.",  # Masked
    "Describe using format - Damage Type: ___; Affected Part: ___; Severity: ___; Notes: ___"  # Format-Guided
]

for prompt in prompts:
    output, time_taken, vram = run_vlm_inference(prompt, filename, df=df)
    inference_outputs.append(output)
    inference_times.append(time_taken)
    vram_usages.append(vram)